In [1]:
import os
import sys
from pathlib import Path
import logging
import time
import shapely
import pandas as pd
import geopandas as gpd
import seaborn
import dask
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

sys.path.insert(0, '..')
import src.hotspot_utils as util 
import src.process_nearest_hotspots as nearest_process

In [2]:
logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(name)s - %(message)s',
    level=logging.DEBUG,
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout,
)
_LOG = logging.getLogger(__name__)

In [3]:
# include n_workers equal or less than the number of core
client = Client(n_workers=8)
client

Client Scheduler: tcp://127.0.0.1:39417 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 16 Memory: 66.57 GB


In [4]:
# This is the output directory outputs and itermediary files from this notebook examples will be stored.
outdir = Path("/home/jovyan/s3vt_dask/s3vtdata/workdir")

# Processing Parameter used in Sub-setting Spatial Extent and Temporal Range for Area of Interest
##### The FRP data from nasa, esa, eumetsat and landgate are merged, sub-setted and neareast hotspots csv files are generated based on the parameters in `processing_parameters`  
##### The parameter `chunks` in blocking FRP data to enable multi-processing. If you encounter memory issues then higher the number.
##### The `start_time` and `end_time` can be used to subset for solar_day (3:00-22:00), solar_night (22:00-03:00 with 12 hours offset) and solar_all(0:00-24:00) hours.

In [5]:
processing_parameters = {
    "nasa_frp": "s3://s3vtaustralia/nasa_hotspots_gdf.geojson",
    "esa_frp": "s3://s3vtaustralia/s3vt_hotspots.geojson",
    "eumetsat_frp": "s3://s3vtaustralia/s3vt_eumetsat_hotspots.geojson",
    "landgate_frp": "s3://s3vtaustralia/landgate_hotspots_gdf.geojson",
    "dea_frp": None,
    "lon_west": 147.0,
    "lat_south": -38.0,
    "lon_east": 154.0,
    "lat_north": -27.,
    "start_date": "2019-11-01",
    "end_date": "2020-10-8",
    "start_time": "20:00",
    "end_time": "03:00",
    "chunks": 500,
    "outdir": outdir,
    "compare_field": "solar_day",
    "swath_config_file": Path("/home/jovyan/s3vtconfig.yaml"),
}

In [6]:
# This is to generate nearest .csv files. If .csv files already exists then skip this process. Takes around ~5-6 hours in this sandbox environment with 2-core and 16 GB RAM
nearest_hotspots_product_files = nearest_process.process_nearest_points(**processing_parameters)

2021-08-17 07:36:03,343: INFO: Processing FRP Hotspots Datasets
2021-08-17 07:36:03,357: INFO: Found credentials in environment variables.
2021-08-17 07:36:03,392: INFO: Fetching FRP datasets...
2021-08-17 07:36:03,393: INFO: s3://s3vtaustralia/nasa_hotspots_gdf.geojson exists: skipped download
2021-08-17 07:36:03,393: INFO: s3://s3vtaustralia/s3vt_hotspots.geojson exists: skipped download
2021-08-17 07:36:03,394: INFO: s3://s3vtaustralia/s3vt_eumetsat_hotspots.geojson exists: skipped download
2021-08-17 07:36:03,394: INFO: s3://s3vtaustralia/landgate_hotspots_gdf.geojson exists: skipped download
2021-08-17 07:36:03,394: INFO: dea Hotspots FRP  is None. excluding from analysis.
2021-08-17 07:36:03,395: INFO: Reading...
2021-08-17 07:36:03,395: INFO: reading and subsetting GeoDataFrame for nasa: /home/jovyan/s3vt_dask/s3vtdata/workdir/nasa_hotspots_gdf.geojson
2021-08-17 07:37:45,277: INFO: reading and subsetting GeoDataFrame for esa: /home/jovyan/s3vt_dask/s3vtdata/workdir/s3vt_hotspot

## Nearest Hotspots DataFrame merged from neareast hotspots csv files

In [7]:
# csv directory is where nearest hotspots csv files are stored. 
csv_directory = outdir
print(csv_directory)
# This is read all the .csv files if name starts with `nearest_points` and ends with `compare_field` value from processing parameters.
nearest_hotspots_csv_files = [
    fp for fp in csv_directory.iterdir()
    if (fp.name.startswith("nearest_points"))
    and (fp.name.endswith("csv"))
]

/home/jovyan/s3vt_dask/s3vtdata/workdir


In [8]:
# nearest points csv files that will be used analysis from here on.
nearest_hotspots_csv_files

[PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_TERRA_MODIS_LANDGATE_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SUOMI NPP_VIIRS_NASA1_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SENTINEL_3B_SLSTR_ESA_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_TERRA_MODIS_NASA6.03_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_NOAA 20_VIIRS_NASA2.0NRT_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SENTINEL_3B_SLSTR_EUMETSAT_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_NOAA 20_VIIRS_LANDGATE_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_SUOMI NPP_VIIRS_LANDGATE_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3vtdata/workdir/nearest_points_AQUA_MODIS_LANDGATE_2000_0300.csv'),
 PosixPath('/home/jovyan/s3vt_dask/s3v

In [9]:
# returns a dask DataFrame with index set at column `compare_field` from processing_parameters.
nearest_points_ddf = util.csv_to_dataframe(nearest_hotspots_csv_files, processing_parameters["compare_field"])

In [10]:
nearest_points_ddf.head()

,Unnamed: 0,latitude,longitude,satellite,sensor,confidence,power,datetime,satellite_sensor_product,geometry,...,2_power,2_datetime,2_solar_day,2_satellite_sensor_product,2_geometry,2_solar_night,dist,dist_m,timedelta,count
solar_day,,,,,,,,,,,,,,,,,,,,,
2019-11-01 22:01:39,53177,-29.873943,153.123579,SENTINEL_3B,SLSTR,-1.0,0.55,2019-11-01 11:49:10,SENTINEL_3B_SLSTR_ESA,POINT (153.1235786134001 -29.87394292457554),...,0.55,2019-11-01 11:49:10,2019-11-01 22:01:39,SENTINEL_3B_SLSTR_ESA,POINT (153.1235786134001 -29.87394292457554),2019-11-01 10:01:39,0.0,0.0,0 days,1
2019-11-01 22:01:39,53179,-29.845804,153.119394,SENTINEL_3B,SLSTR,-1.0,0.32,2019-11-01 11:49:11,SENTINEL_3B_SLSTR_ESA,POINT (153.1193936134107 -29.84580392464659),...,0.32,2019-11-01 11:49:11,2019-11-01 22:01:39,SENTINEL_3B_SLSTR_ESA,POINT (153.1193936134107 -29.84580392464659),2019-11-01 10:01:39,0.0,0.0,0 days,1
2019-11-01 22:01:39,53178,-29.863303,153.123948,SENTINEL_3B,SLSTR,-1.0,0.46,2019-11-01 11:49:10,SENTINEL_3B_SLSTR_ESA,POINT (153.1239476133992 -29.86330292460241),...,0.46,2019-11-01 11:49:10,2019-11-01 22:01:39,SENTINEL_3B_SLSTR_ESA,POINT (153.1239476133992 -29.86330292460241),2019-11-01 10:01:39,0.0,0.0,0 days,1
2019-11-01 22:01:39,53176,-29.845804,153.119394,SENTINEL_3B,SLSTR,-1.0,0.32,2019-11-01 11:49:11,SENTINEL_3B_SLSTR_ESA,POINT (153.1193936134107 -29.84580392464659),...,0.32,2019-11-01 11:49:11,2019-11-01 22:01:39,SENTINEL_3B_SLSTR_ESA,POINT (153.1193936134107 -29.84580392464659),2019-11-01 10:01:39,0.0,0.0,0 days,1
2019-11-01 22:01:39,53175,-29.863303,153.123948,SENTINEL_3B,SLSTR,-1.0,0.46,2019-11-01 11:49:10,SENTINEL_3B_SLSTR_ESA,POINT (153.1239476133992 -29.86330292460241),...,0.46,2019-11-01 11:49:10,2019-11-01 22:01:39,SENTINEL_3B_SLSTR_ESA,POINT (153.1239476133992 -29.86330292460241),2019-11-01 10:01:39,0.0,0.0,0 days,1


# Results
## Co-occurrence metrics

In [11]:
region_alias = "nsw"
output_directory = processing_parameters["outdir"]
comparison_prefix = (
    f"{processing_parameters['start_date'].replace('-', '')}"
    f"_{processing_parameters['end_date'].replace('-', '')}"
    f"_{processing_parameters['start_time'].replace(':','')}"
    f"_{processing_parameters['end_time'].replace(':','')}"
    f"_{region_alias}"
)

In [12]:
# set the nearest distance threshold between two hotspots to confine the analysis within the distance threshold.  
dist_threshold = 5000  # units in meters

In [13]:
nearest_ddf_dist_subset = client.persist(nearest_points_ddf[nearest_points_ddf["dist_m"] < dist_threshold])

In [14]:
# Count of hotspot matches < dist_threshold
numerator = util.dask_pivot_table(
    nearest_ddf_dist_subset,
    index="2_satellite_sensor_product",
    column="satellite_sensor_product",
    values="count",
    aggfunc="count"
).compute()

In [15]:
numerator.astype(int).to_csv(output_directory.joinpath(f"{comparison_prefix}_matches_{dist_threshold}.csv"))
numerator.astype(int)

satellite_sensor_product,SENTINEL_3B_SLSTR_ESA,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_LANDGATE,TERRA_MODIS_NASA6.03,SUOMI NPP_VIIRS_NASA1,SUOMI NPP_VIIRS_LANDGATE,NOAA 20_VIIRS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_EUMETSAT,NOAA 20_VIIRS_NASA2.0NRT
2_satellite_sensor_product,,,,,,,,,,,,
SENTINEL_3B_SLSTR_ESA,52762,8400,22466,5832,8451,1138,5604,54,17,36434,5045,3176
SENTINEL_3A_SLSTR_ESA,14149,39089,23496,6176,14522,982,1361,32,11,7862,23248,1912
TERRA_MODIS_NASA6.03,20027,15193,61342,17196,538,318,5514,32,11,14542,9579,3745
TERRA_MODIS_LANDGATE,23435,16398,64494,17116,566,451,6282,32,11,17747,10651,4334
SUOMI NPP_VIIRS_LANDGATE,235,475,144,35,15809,27065,0,0,0,139,331,0
SUOMI NPP_VIIRS_NASA1,5227,4905,194,53,39527,24652,0,0,0,2723,2270,0
NOAA 20_VIIRS_LANDGATE,2104,576,3071,890,0,0,12168,0,0,1008,615,3690
AQUA_MODIS_LANDGATE,20,7,24,8,0,0,0,87,25,0,5,0
AQUA_MODIS_NASA6.03,20,7,24,8,0,0,0,87,25,0,5,0


In [16]:
# Count of hotspot matches - total  
denominator = util.dask_pivot_table(
        nearest_points_ddf,
        index="2_satellite_sensor_product",
        column="satellite_sensor_product",
        values="count",
        aggfunc="count",
    ).compute()

In [17]:
denominator.astype(int).to_csv(output_directory.joinpath(f"{comparison_prefix}_matches_count.csv"))
denominator.astype(int)

satellite_sensor_product,SENTINEL_3B_SLSTR_ESA,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_LANDGATE,TERRA_MODIS_NASA6.03,SUOMI NPP_VIIRS_NASA1,SUOMI NPP_VIIRS_LANDGATE,NOAA 20_VIIRS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_EUMETSAT,NOAA 20_VIIRS_NASA2.0NRT
2_satellite_sensor_product,,,,,,,,,,,,
SENTINEL_3B_SLSTR_ESA,52762,19171,30715,8404,13968,6973,8992,61,19,36456,13277,9731
SENTINEL_3A_SLSTR_ESA,15667,39089,34942,8740,24095,11100,2603,87,25,8607,23268,3418
TERRA_MODIS_NASA6.03,28007,18112,63773,17196,694,377,10332,87,25,21580,11613,12424
TERRA_MODIS_LANDGATE,28245,18227,64494,17195,791,654,11856,87,25,22110,12450,13349
SUOMI NPP_VIIRS_LANDGATE,2650,4080,269,69,27231,27065,0,0,0,2196,2542,0
SUOMI NPP_VIIRS_NASA1,6631,6048,267,74,39527,24775,0,0,0,3785,2928,0
NOAA 20_VIIRS_LANDGATE,4373,1649,9774,3056,0,0,12168,0,0,2983,1521,11583
AQUA_MODIS_LANDGATE,20,7,24,8,0,0,0,87,25,0,5,0
AQUA_MODIS_NASA6.03,20,7,24,8,0,0,0,87,25,0,5,0


In [18]:
# Difference of matched points closer than 5000m
difference = denominator - numerator

In [19]:
difference.astype(int).to_csv(output_directory.joinpath(f"{comparison_prefix}_count_difference.csv"))
difference.astype(int)

satellite_sensor_product,SENTINEL_3B_SLSTR_ESA,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_LANDGATE,TERRA_MODIS_NASA6.03,SUOMI NPP_VIIRS_NASA1,SUOMI NPP_VIIRS_LANDGATE,NOAA 20_VIIRS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_EUMETSAT,NOAA 20_VIIRS_NASA2.0NRT
2_satellite_sensor_product,,,,,,,,,,,,
SENTINEL_3B_SLSTR_ESA,0,10771,8249,2572,5517,5835,3388,7,2,22,8232,6555
SENTINEL_3A_SLSTR_ESA,1518,0,11446,2564,9573,10118,1242,55,14,745,20,1506
TERRA_MODIS_NASA6.03,7980,2919,2431,0,156,59,4818,55,14,7038,2034,8679
TERRA_MODIS_LANDGATE,4810,1829,0,79,225,203,5574,55,14,4363,1799,9015
SUOMI NPP_VIIRS_LANDGATE,2415,3605,125,34,11422,0,0,0,0,2057,2211,0
SUOMI NPP_VIIRS_NASA1,1404,1143,73,21,0,123,0,0,0,1062,658,0
NOAA 20_VIIRS_LANDGATE,2269,1073,6703,2166,0,0,0,0,0,1975,906,7893
AQUA_MODIS_LANDGATE,0,0,0,0,0,0,0,0,0,0,0,0
AQUA_MODIS_NASA6.03,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
# Percentage of matched points closer than dist_threshold
percentage = (numerator / denominator) * 100
percentage = np.round(percentage, 2)

In [21]:
percentage.to_csv(output_directory.joinpath(f"{comparison_prefix}_percentage.csv"))
percentage

satellite_sensor_product,SENTINEL_3B_SLSTR_ESA,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_LANDGATE,TERRA_MODIS_NASA6.03,SUOMI NPP_VIIRS_NASA1,SUOMI NPP_VIIRS_LANDGATE,NOAA 20_VIIRS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_EUMETSAT,NOAA 20_VIIRS_NASA2.0NRT
2_satellite_sensor_product,,,,,,,,,,,,
SENTINEL_3B_SLSTR_ESA,100.00,43.82,73.14,69.40,60.50,16.32,62.32,88.52,89.47,99.94,38.00,32.64
SENTINEL_3A_SLSTR_ESA,90.31,100.00,67.24,70.66,60.27,8.85,52.29,36.78,44.00,91.34,99.91,55.94
TERRA_MODIS_NASA6.03,71.51,83.88,96.19,100.00,77.52,84.35,53.37,36.78,44.00,67.39,82.49,30.14
TERRA_MODIS_LANDGATE,82.97,89.97,100.00,99.54,71.55,68.96,52.99,36.78,44.00,80.27,85.55,32.47
SUOMI NPP_VIIRS_LANDGATE,8.87,11.64,53.53,50.72,58.06,100.00,NaN,NaN,NaN,6.33,13.02,NaN
SUOMI NPP_VIIRS_NASA1,78.83,81.10,72.66,71.62,100.00,99.50,NaN,NaN,NaN,71.94,77.53,NaN
NOAA 20_VIIRS_LANDGATE,48.11,34.93,31.42,29.12,NaN,NaN,100.00,NaN,NaN,33.79,40.43,31.86
AQUA_MODIS_LANDGATE,100.00,100.00,100.00,100.00,NaN,NaN,NaN,100.00,100.00,NaN,100.00,NaN
AQUA_MODIS_NASA6.03,100.00,100.00,100.00,100.00,NaN,NaN,NaN,100.00,100.00,NaN,100.00,NaN


In [22]:
# Maximum time between match points < dist_threshold
timemax = util.pandas_pivot_table(
    nearest_ddf_dist_subset.compute(),
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["timedelta"],
    aggfunc={"timedelta": np.max}
    
)

In [23]:
timemax.to_csv(output_directory.joinpath(f"{comparison_prefix}_max_time_matched_points.csv"))
timemax

timedelta                      \
2_satellite_sensor_product AQUA_MODIS_LANDGATE AQUA_MODIS_NASA6.03   
satellite_sensor_product                                             
AQUA_MODIS_LANDGATE                   00:01:00            00:02:00   
AQUA_MODIS_NASA6.03                   00:02:00            00:00:00   
NOAA 20_VIIRS_LANDGATE                     NaT                 NaT   
NOAA 20_VIIRS_NASA2.0NRT                   NaT                 NaT   
SENTINEL_3A_SLSTR_ESA                 21:46:45            21:44:45   
SENTINEL_3A_SLSTR_EUMETSAT            21:16:28            21:14:28   
SENTINEL_3B_SLSTR_ESA                 21:07:13            21:05:13   
SENTINEL_3B_SLSTR_EUMETSAT                 NaT                 NaT   
SUOMI NPP_VIIRS_LANDGATE                   NaT                 NaT   
SUOMI NPP_VIIRS_NASA1                      NaT                 NaT   
TERRA_MODIS_LANDGATE                  21:40:00            21:38:00   
TERRA_MODIS_NASA6.03                  21:34:00            21:32:00   

                                                                            \
2_satellite_sensor_product NOAA 20_VIIRS_LANDGATE NOAA 20_VIIRS_NASA2.0NRT   
satellite_sensor_product                                                     
AQUA_MODIS_LANDGATE                           NaT                      NaT   
AQUA_MODIS_NASA6.03                           NaT                      NaT   
NOAA 20_VIIRS_LANDGATE                   01:43:00                 22:06:00   
NOAA 20_VIIRS_NASA2.0NRT                 22:06:00                 00:00:00   
SENTINEL_3A_SLSTR_ESA                    22:24:15                 22:30:18   
SENTINEL_3A_SLSTR_EUMETSAT               22:24:15                 22:33:47   
SENTINEL_3B_SLSTR_ESA                    22:03:13                 22:33:52   
SENTINEL_3B_SLSTR_EUMETSAT               22:22:19                 22:33:52   
SUOMI NPP_VIIRS_LANDGATE                      NaT                      NaT   
SUOMI NPP_VIIRS_NASA1                         NaT                      NaT   
TERRA_MODIS_LANDGATE                     22:33:00                 22:33:00   
TERRA_MODIS_NASA6.03                     22:24:00                 22:25:00   

                                                                             \
2_satellite_sensor_product SENTINEL_3A_SLSTR_ESA SENTINEL_3A_SLSTR_EUMETSAT   
satellite_sensor_product                                                      
AQUA_MODIS_LANDGATE                     21:46:45                   21:16:28   
AQUA_MODIS_NASA6.03                     21:44:45                   21:14:28   
NOAA 20_VIIRS_LANDGATE                  22:24:15                   22:24:15   
NOAA 20_VIIRS_NASA2.0NRT                22:30:18                   22:33:46   
SENTINEL_3A_SLSTR_ESA                   00:00:01                   00:00:01   
SENTINEL_3A_SLSTR_EUMETSAT              00:00:01                   00:00:01   
SENTINEL_3B_SLSTR_ESA                   00:40:07                   00:40:06   
SENTINEL_3B_SLSTR_EUMETSAT              00:40:06                   00:40:06   
SUOMI NPP_VIIRS_LANDGATE                22:16:45                   22:16:45   
SUOMI NPP_VIIRS_NASA1                   22:24:25                   22:21:52   
TERRA_MODIS_LANDGATE                    22:39:42                   22:39:42   
TERRA_MODIS_NASA6.03                    01:10:09                   01:10:09   

                                                                             \
2_satellite_sensor_product SENTINEL_3B_SLSTR_ESA SENTINEL_3B_SLSTR_EUMETSAT   
satellite_sensor_product                                                      
AQUA_MODIS_LANDGATE                     21:07:13                        NaT   
AQUA_MODIS_NASA6.03                     21:05:13                        NaT   
NOAA 20_VIIRS_LANDGATE                  22:03:13                   22:22:19   
NOAA 20_VIIRS_NASA2.0NRT                22:33:52                   22:33:52   
SENTINEL_3A_SLSTR_ESA                   00:40:07                   00:40:07 

In [24]:
# Minimum time between match points < dist_threshold
timemin = util.pandas_pivot_table(
    nearest_ddf_dist_subset.compute(),
    index=["satellite_sensor_product"],
    columns=["2_satellite_sensor_product"],
    values=["timedelta"],
    aggfunc={"timedelta": np.min}
    
)

In [25]:
timemin.to_csv(output_directory.joinpath(f"{comparison_prefix}_min_time_matched_points.csv"))
timemin

timedelta                      \
2_satellite_sensor_product AQUA_MODIS_LANDGATE AQUA_MODIS_NASA6.03   
satellite_sensor_product                                             
AQUA_MODIS_LANDGATE                   00:00:00            00:01:00   
AQUA_MODIS_NASA6.03                   00:01:00            00:00:00   
NOAA 20_VIIRS_LANDGATE                     NaT                 NaT   
NOAA 20_VIIRS_NASA2.0NRT                   NaT                 NaT   
SENTINEL_3A_SLSTR_ESA                 21:15:28            21:14:28   
SENTINEL_3A_SLSTR_EUMETSAT            21:15:28            21:14:28   
SENTINEL_3B_SLSTR_ESA                 21:07:05            21:05:05   
SENTINEL_3B_SLSTR_EUMETSAT                 NaT                 NaT   
SUOMI NPP_VIIRS_LANDGATE                   NaT                 NaT   
SUOMI NPP_VIIRS_NASA1                      NaT                 NaT   
TERRA_MODIS_LANDGATE                  21:25:00            21:24:00   
TERRA_MODIS_NASA6.03                  21:33:00            21:32:00   

                                                                            \
2_satellite_sensor_product NOAA 20_VIIRS_LANDGATE NOAA 20_VIIRS_NASA2.0NRT   
satellite_sensor_product                                                     
AQUA_MODIS_LANDGATE                           NaT                      NaT   
AQUA_MODIS_NASA6.03                           NaT                      NaT   
NOAA 20_VIIRS_LANDGATE                   00:00:00                 00:00:00   
NOAA 20_VIIRS_NASA2.0NRT                 00:00:00                 00:00:00   
SENTINEL_3A_SLSTR_ESA                    19:59:28                 01:05:42   
SENTINEL_3A_SLSTR_EUMETSAT               01:24:59                 01:05:42   
SENTINEL_3B_SLSTR_ESA                    20:04:14                 01:12:20   
SENTINEL_3B_SLSTR_EUMETSAT               20:09:58                 01:12:20   
SUOMI NPP_VIIRS_LANDGATE                      NaT                      NaT   
SUOMI NPP_VIIRS_NASA1                         NaT                      NaT   
TERRA_MODIS_LANDGATE                     20:08:00                 01:35:00   
TERRA_MODIS_NASA6.03                     20:14:00                 01:29:00   

                                                                             \
2_satellite_sensor_product SENTINEL_3A_SLSTR_ESA SENTINEL_3A_SLSTR_EUMETSAT   
satellite_sensor_product                                                      
AQUA_MODIS_LANDGATE                     21:15:28                   21:15:28   
AQUA_MODIS_NASA6.03                     21:14:28                   21:14:28   
NOAA 20_VIIRS_LANDGATE                  19:57:28                   01:24:59   
NOAA 20_VIIRS_NASA2.0NRT                01:05:42                   01:05:42   
SENTINEL_3A_SLSTR_ESA                   00:00:00                   00:00:00   
SENTINEL_3A_SLSTR_EUMETSAT              00:00:00                   00:00:00   
SENTINEL_3B_SLSTR_ESA                   00:38:49                   00:38:49   
SENTINEL_3B_SLSTR_EUMETSAT              00:38:49                   00:38:49   
SUOMI NPP_VIIRS_LANDGATE                20:06:49                   20:06:13   
SUOMI NPP_VIIRS_NASA1                   20:04:49                   20:14:09   
TERRA_MODIS_LANDGATE                    00:00:04                   00:00:09   
TERRA_MODIS_NASA6.03                    00:00:01                   00:00:01   

                                                                             \
2_satellite_sensor_product SENTINEL_3B_SLSTR_ESA SENTINEL_3B_SLSTR_EUMETSAT   
satellite_sensor_product                                                      
AQUA_MODIS_LANDGATE                     21:07:05                        NaT   
AQUA_MODIS_NASA6.03                     21:05:05                        NaT   
NOAA 20_VIIRS_LANDGATE                  20:04:14                   20:09:30   
NOAA 20_VIIRS_NASA2.0NRT                01:12:20                   01:12:20   
SENTINEL_3A_SLSTR_ESA                   00:38:49                   00:38:49 

In [26]:
# Average distance (m) between matched points < dist_threshold
averagedist = util.dask_pivot_table(
    nearest_ddf_dist_subset,
    index="2_satellite_sensor_product",
    column="satellite_sensor_product",
    values="dist_m",
    aggfunc="mean",
).compute()

In [27]:
averagedist = np.round(averagedist, 2)
averagedist.to_csv(output_directory.joinpath(f"{comparison_prefix}_average_distance_{dist_threshold}m.csv"))
averagedist

satellite_sensor_product,SENTINEL_3B_SLSTR_ESA,SENTINEL_3A_SLSTR_ESA,TERRA_MODIS_LANDGATE,TERRA_MODIS_NASA6.03,SUOMI NPP_VIIRS_NASA1,SUOMI NPP_VIIRS_LANDGATE,NOAA 20_VIIRS_LANDGATE,AQUA_MODIS_LANDGATE,AQUA_MODIS_NASA6.03,SENTINEL_3B_SLSTR_EUMETSAT,SENTINEL_3A_SLSTR_EUMETSAT,NOAA 20_VIIRS_NASA2.0NRT
2_satellite_sensor_product,,,,,,,,,,,,
SENTINEL_3B_SLSTR_ESA,0.00,756.40,693.45,577.63,770.44,606.02,821.64,1583.41,1920.05,3.20,812.78,888.63
SENTINEL_3A_SLSTR_ESA,1068.69,0.00,615.00,525.38,1096.74,649.79,808.92,902.79,1553.39,1098.71,1.92,712.97
TERRA_MODIS_NASA6.03,1620.97,1237.24,540.10,0.00,1838.30,1313.46,1387.11,997.92,1795.03,1679.59,1141.55,1346.33
TERRA_MODIS_LANDGATE,1312.13,888.96,0.00,127.32,1212.40,984.93,1078.19,615.53,1077.73,1369.06,836.00,1087.70
SUOMI NPP_VIIRS_LANDGATE,1293.81,1719.65,1589.29,1853.68,505.06,0.00,NaN,NaN,NaN,808.66,1430.20,NaN
SUOMI NPP_VIIRS_NASA1,1243.83,965.65,1275.25,1438.15,0.00,224.50,NaN,NaN,NaN,1305.07,1041.63,NaN
NOAA 20_VIIRS_LANDGATE,1657.90,1457.32,1577.60,1561.01,NaN,NaN,0.00,NaN,NaN,1676.16,1366.92,617.70
AQUA_MODIS_LANDGATE,1107.30,508.48,636.24,949.88,NaN,NaN,NaN,0.00,233.26,NaN,310.25,NaN
AQUA_MODIS_NASA6.03,1479.30,592.61,1059.23,1136.57,NaN,NaN,NaN,655.05,0.00,NaN,312.60,NaN


In [ ]:
client.close() # close dask.distributed client